# 在上一个代码的基础上，加一些注释，修改数据获取的方式

In [1]:
import tensorflow as tf 
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import models, Model
from tensorflow import keras
from tensorflow.keras import layers, Sequential, datasets, optimizers
import matplotlib.pyplot as plt
import numpy as np 
import glob
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Global Variables
BATCHSZ = 32
EPOCHS = 100

In [4]:
data_dir= "E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D"
data_root = glob.glob(data_dir + '/*')
for name in glob.glob(data_dir + '/*'):
    print(name)
train = np.load(data_root[4])
train_label = np.load(data_root[5])
test = np.load(data_root[2])
test_label = np.load(data_root[3])
train.shape, train_label.shape, test.shape, test_label.shape

E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train_label.npy


((3200, 9, 9, 204), (3200,), (50929, 9, 9, 204), (50929,))

In [5]:
class_num = 16
im_height = 9
im_width = 9
im_channel = train.shape[3]
train_num = train.shape[0]
val_num = test.shape[0]

# 创建dataset

In [6]:
# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [9]:
# load train dataset
db_train = tf.data.Dataset.from_tensor_slices((train, train_label))
# train_dataset = train_dataset.shuffle(buffer_size=train_num).repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
db_train = db_train.shuffle(buffer_size=train_num).batch(BATCHSZ).prefetch(AUTOTUNE)

# load test dataset
db_test = tf.data.Dataset.from_tensor_slices((test, test_label))
# val_dataset = val_dataset.repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
db_test = db_test.batch(BATCHSZ).prefetch(AUTOTUNE)

In [10]:
db_train, db_test

(<PrefetchDataset shapes: ((None, 9, 9, 204), (None,)), types: (tf.float32, tf.int32)>,
 <PrefetchDataset shapes: ((None, 9, 9, 204), (None,)), types: (tf.float32, tf.int32)>)

# 构建 ResNet

- add（）：直接对张量求和,add层将dense_1层的输入和dense_2层的输入加在了一起，是张量元素内容相加。
- conatenate（）：串联一个列表的输入张量。对一维进行了串联，通道数变成了x + x = 2x，可以指指定 axis = x 来指定空间的第 x 维串联。

In [11]:
# 实现残差块
# (3,3) (1,1) 是卷积核的大小

class BasicBlock(layers.Layer):

    def __init__(self, filter_num, stride=1):   
        super(BasicBlock, self).__init__()

        # padding 对于能够整除的四周均匀补全，对于不能整除的，自适应补全，保证能够用到原始图像的全部像素信息，
        # 并不是保证 padding 后图像大小一致，当 stride = 2， 使用 padding = same时，必使结果的图像大小变为一半
        # “第一层” strides=stride，有时进行下采样（stride > 1）,有时不进行下采样
        self.conv1 = layers.Conv2D(filter_num, (3,3), strides=stride, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')

        # “第二层”
        self.conv2 = layers.Conv2D(filter_num, (3,3), strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        
        # "分支层"
        # 保证从 x 直接连到下面的线两端能够之间相加，如果上一个 Residual Block 的输出维度和当前的 Residual Block 的维度不一样，
        # 那就对这个 x 进行 downSample 操作，使得维度一致
        if stride != 1:
            # 短接层， identity layer，这里的 strides 与第一层的 strides 相同，保证结果可以和第二层直接相加
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
        else:
            # 如果 strides = 1，保证结果可以和第二层直接相加，就不需要 downsample
            self.downsample = lambda x:x

    # 残差块内正向传播过程，包含两个卷积层
    def call(self, inputs, training=None):
        # 前向传播
        # [b, h ,w, c]
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        # 此处调用 downsample  有两种形式，看stride 的值而定
        identity = self.downsample(inputs)

        output = layers.add([out, identity])    # 这里的相加是对应元素相加
        output = self.relu(output)              # 没有参数的层可以定义一个层用两次

        return output

In [12]:
# ResNet 是多个 BasicBlock 顿叠而成
class ResNet(keras.Model):
    
    # layer_dims [2,2,2,2]
    def __init__(self, layer_dims, num_calsses=16):   # layer_dims [2,2,2,2] 每一层的basic block个数
        super(ResNet, self).__init__()
        
        # 设置预处理层
        self.stem = Sequential([layers.Conv2D(64, (3,3), strides=(1, 1)),
                               layers.BatchNormalization(),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size=(2,2), strides=(1, 1), padding='same')
                               ])

        # 中间层
        self.layers1 = self.build_resblock(64, layer_dims[0])   # 调用数组layer_dims中的第 0 维定义的 basicBlock个数
        
        # h, w 维会变小，这里stride 等于 2，使得 feature size 越来越小，channel 越来越多
        self.layers2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layers3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layers4 = self.build_resblock(512, layer_dims[3], stride=2)

        # 分类层
        # 全连接层 output : [b, 512, h, w]，自适应输出用于输出
        self.avgpool = layers.GlobalAveragePooling2D()  # 具体大小为6 × 6 × 3，经过GAP转换后，变成了大小为 1 × 1 × 3 的输出值，每一层 h × w 会被平均化成一个值
        self.fc = layers.Dense(16)  # TODO


    def call(self, inputs, training=None):
        # 前向运算过程，预处理
        x = self.stem(inputs)
        # 4 个 resBlock
        x = self.layers1(x)
        x = self.layers2(x)
        x = self.layers3(x)
        x = self.layers4(x)

        # [b, c]
        x = self.avgpool(x)
        # [b, 16]
        x = self.fc(x)
        return x
    
    # 实现 resblock
    # 创建一个resBlock， 一个 resBlock 中包含多个 basicBlock
    def build_resblock(self, filter_num, blocks, stride=1):    # 通道数，ResNet 会堆叠多少，默认步长为 1
        res_blocks = Sequential()
        # 添加第一层， may down sample
        res_blocks.add(BasicBlock(filter_num, stride))
        # 后面的 BasicBlock 不会下采样，因为此处定义了 stride 为 1
        for _ in range(1, blocks):
            res_blocks.add(BasicBlock(filter_num, stride=1))
        return res_blocks

- 传给类中的参数，都被 \__init__ ()中的形式变量接收了
- 因为类中已经实现，\__call__() 魔法方法,因此不用外部调用就可以运行build()、call()等函数？

In [13]:
def resnet18():
    return ResNet([2, 2, 2, 2])   #   1 + 4 * 4 + 1

In [14]:
sample = next(iter(db_train))
print(sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

(32, 9, 9, 204) (32,) tf.Tensor(0.000542417, shape=(), dtype=float32) tf.Tensor(0.90160555, shape=(), dtype=float32)


In [19]:
def main():
    model = resnet18()
    model.build(input_shape=(None, 9, 9, 204))  # TODO 需要修改
    model.summary()
    optimizer = optimizers.Adam(lr=1e-3)
    
    for epoch in range(30):
        for step, (x, y) in enumerate(db_train):
            with tf.GradientTape() as tape:
                # [b, 32, 32, 3]  =>  [b, 100]
                logits = model(x)
                # print(logits.shape)
                # [b] => [b, 100]
                y_onthot = tf.one_hot(y, depth=class_num)         # TODO 需要修改
                # print(y_onthot.shape) 
                loss = tf.losses.categorical_crossentropy(y_onthot, logits, from_logits=True)
                loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            if step % 300 ==0:
                print(epoch, step, 'loss', float(loss))

        total_num = 0
        total_correct = 0

        for x, y in db_test:
            logits = model(x)
            prob = tf.nn.softmax(logits, axis=1)
            pred = tf.argmax(prob, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)

            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)
            
            total_num += x.shape[0]
            
            total_correct += int(correct)

        acc = total_correct / total_num
        print(epoch, 'acc', acc)

In [20]:
main()

Model: "res_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_16 (Sequential)   (None, 7, 7, 64)          117824    
_________________________________________________________________
sequential_17 (Sequential)   (None, 7, 7, 64)          148480    
_________________________________________________________________
sequential_18 (Sequential)   (None, 4, 4, 128)         526464    
_________________________________________________________________
sequential_20 (Sequential)   (None, 2, 2, 256)         2101504   
_________________________________________________________________
sequential_22 (Sequential)   (None, 1, 1, 512)         8397312   
_________________________________________________________________
global_average_pooling2d_2 ( multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  82